In [183]:
import pandas as pd
import numpy as np

In [184]:
! git clone https://github.com/andrealodi/ORIE-5213-Spring2023.git

31407.65s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


fatal: destination path 'ORIE-5213-Spring2023' already exists and is not an empty directory.


In [185]:
initial_inven= pd.read_json("ORIE-5213-Spring2023/Bike sharing/Dataset/Initial_Inven.json")

In [186]:
initial_inven

,0
0,7
1,0
2,1
3,9
4,0
5,9
6,5
7,18
8,3
9,5


In [187]:
# number of bikes initially at stand 20
initial_inven.iloc[20]

0    13
Name: 20, dtype: int64

In [188]:
simu0= pd.read_json("ORIE-5213-Spring2023/Bike sharing/Dataset/simu0_0.json")

In [189]:
simu0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       1458 non-null   int64
 1   1       1458 non-null   int64
 2   2       1458 non-null   int64
 3   3       1458 non-null   int64
dtypes: int64(4)
memory usage: 45.7 KB


In [190]:
simu0.describe()

,0,1,2,3
count,1458.000000,1458.000000,1458.000000,1458.000000
mean,834.290123,14.154321,846.270919,13.951989
std,293.686675,9.045841,295.914786,9.105823
min,4.000000,0.000000,1.000000,0.000000
25%,553.750000,6.000000,565.250000,5.000000
50%,903.500000,15.000000,911.500000,14.000000
75%,1060.750000,22.000000,1077.000000,22.000000
max,1439.000000,29.000000,1429.000000,29.000000


In [194]:
simu0

,0,1,2,3
0,4,1,22,11
1,15,1,37,18
2,19,12,45,8
3,48,11,72,5
4,50,22,73,27
...,...,...,...,...
1453,1414,27,1443,9
1454,1417,17,1428,3
1455,1432,0,1454,8
1456,1439,9,1461,29


In [195]:
# Adjust column 2 based on values in column 0
simu0.loc[simu0[0] > simu0[2], 2] += 1440


In [196]:
simu0

,0,1,2,3
0,4,1,22,11
1,15,1,37,18
2,19,12,45,8
3,48,11,72,5
4,50,22,73,27
...,...,...,...,...
1453,1414,27,1443,9
1454,1417,17,1428,3
1455,1432,0,1454,8
1456,1439,9,1461,29


In [197]:
import pandas as pd

# Define the time ranges
time_range = [(i, i + 15) for i in range(0, 1440, 15)]

# Create the function to assign the time range
def assign_time_range(value):
    for start, end in time_range:
        if start <= value < end:
            return f"{start}-{end}"
    return "Unknown"

# Create the 'departure_time' column
simu0['departure_time'] = simu0[0].apply(assign_time_range)
simu0['arrival_time'] = simu0[2].apply(assign_time_range)

In [198]:
simu0.groupby([1,'departure_time'])[0].count().reset_index()


,1,departure_time,0
0,0,1005-1020,5
1,0,1020-1035,6
2,0,1035-1050,3
3,0,1050-1065,3
4,0,1065-1080,2
...,...,...,...
948,29,855-870,1
949,29,915-930,1
950,29,930-945,2
951,29,960-975,3


In [199]:
df_rentals = simu0.groupby([1, 'departure_time'])[0].count().reset_index()
df_rentals = df_rentals.rename(columns={0:"rentals", 1: "station_id"})
df_rentals

,station_id,departure_time,rentals
0,0,1005-1020,5
1,0,1020-1035,6
2,0,1035-1050,3
3,0,1050-1065,3
4,0,1065-1080,2
...,...,...,...
948,29,855-870,1
949,29,915-930,1
950,29,930-945,2
951,29,960-975,3


In [200]:
# df_rentals.value_counts()

In [201]:
df_returns = simu0.groupby([3, 'arrival_time'])[2].count().reset_index()
df_returns = df_returns.rename(columns={2:"returns", 3: "station_id"})
df_returns

,station_id,arrival_time,returns
0,0,1065-1080,1
1,0,1080-1095,1
2,0,1095-1110,1
3,0,1125-1140,1
4,0,1140-1155,2
...,...,...,...
952,29,945-960,1
953,29,960-975,3
954,29,975-990,1
955,29,990-1005,1


In [202]:
import os
import pandas as pd

folder_path = "ORIE-5213-Spring2023/Bike sharing/Dataset"
json_files = [file for file in os.listdir(folder_path) if (file.endswith(".json") & file.startswith("simu"))]

dataframes = []
for json_file in json_files:
    day = json_file.split("_")[1].split(".")[0]
    file_path = os.path.join(folder_path, json_file)
    df = pd.read_json(file_path)
    df["day"] = int(day)
    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True).sort_values(by='day')


In [203]:
combined_df["rand"] = np.random.choice(1000, len(combined_df))
combined_df.loc[combined_df["rand"] >= 400, "bike_type"] = 'classic'
combined_df.loc[combined_df["rand"] < 400, "bike_type"] = 'electric'
combined_df.drop(['rand'], axis=1, inplace=True)

In [204]:
combined_df.head(20)

,0,1,2,3,day,bike_type
22727,861,23,880,11,0,classic
23012,1026,2,1054,29,0,electric
23011,1026,1,1043,21,0,classic
23010,1026,1,1041,21,0,electric
23009,1025,14,1034,28,0,classic
23008,1025,4,1052,24,0,classic
23007,1025,3,1050,9,0,classic
23006,1025,3,1039,16,0,electric
23005,1024,28,1045,3,0,electric
23013,1027,8,1036,19,0,classic


In [205]:
combined_df['dt'] = ""
combined_df['at'] = ""

for index, row in combined_df.iterrows():
    if row['day'] != 0:
        dt = row[0] + (1440 * row['day'])
        at = row[2] + (1440 * row['day'])
    else:
        dt = row[0]
        at = row[2]
    combined_df.at[index, 'dt'] = dt
    combined_df.at[index, 'at'] = at


In [206]:
# pd.set_option('display.max_rows', None)

In [207]:
combined_df[(combined_df['day']==0) & (combined_df[2] < combined_df[0])]

,0,1,2,3,day,bike_type,dt,at
23491,1439,9,21,29,0,electric,1439,21
23490,1432,0,14,8,0,electric,1432,14
23488,1414,27,3,9,0,classic,1414,3
23487,1412,21,1,9,0,classic,1412,1
23492,1439,16,7,17,0,classic,1439,7


In [208]:
# Adjust column 2 based on values in column 0
combined_df.loc[combined_df[0] > combined_df[2], 2] += 1440


In [209]:
combined_df[(combined_df['day']==0) & (combined_df[2] < combined_df[0])]

,0,1,2,3,day,bike_type,dt,at


In [210]:
import pandas as pd

# Define the time ranges
time_range = [(i, i + 15) for i in range(0, 1440, 15)]

# Create the function to assign the time range
def assign_time_range(value):
    for start, end in time_range:
        if start <= value < end:
            return f"{start}-{end}"
    return "1425-1440"

# Create the 'departure_time' column
combined_df['departure_time'] = combined_df[0].apply(assign_time_range)
combined_df['arrival_time'] = combined_df[2].apply(assign_time_range)

In [211]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 732539 entries, 22727 to 27871
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   0               732539 non-null  int64 
 1   1               732539 non-null  int64 
 2   2               732539 non-null  int64 
 3   3               732539 non-null  int64 
 4   day             732539 non-null  int64 
 5   bike_type       732539 non-null  object
 6   dt              732539 non-null  object
 7   at              732539 non-null  object
 8   departure_time  732539 non-null  object
 9   arrival_time    732539 non-null  object
dtypes: int64(5), object(5)
memory usage: 77.6+ MB


In [212]:
df_rentals = combined_df.groupby([1, 'departure_time', 'day'])[0].count().reset_index()
df_rentals = df_rentals.rename(columns={0:"rentals", 1: "station_id"})
df_rentals.head(20)

,station_id,departure_time,day,rentals
0,0,0-15,2,2
1,0,0-15,6,1
2,0,0-15,16,1
3,0,0-15,33,1
4,0,0-15,36,1
5,0,0-15,51,1
6,0,0-15,63,1
7,0,0-15,76,1
8,0,0-15,87,1
9,0,0-15,95,1


In [213]:
df_returns = combined_df.groupby([3, 'arrival_time', 'day'])[2].count().reset_index()
df_returns = df_returns.rename(columns={2:"returns", 3: "station_id"})
df_returns.head(20)

,station_id,arrival_time,day,returns
0,0,0-15,14,1
1,0,0-15,61,1
2,0,0-15,162,1
3,0,0-15,184,1
4,0,0-15,260,1
5,0,0-15,311,1
6,0,0-15,329,1
7,0,0-15,337,1
8,0,0-15,372,1
9,0,0-15,407,1


In [214]:
def calculate_time_period(row):
    day = row['day']
    departure_time = int(row['departure_time'].split('-')[0])
    time_period = (day * 96) + (departure_time // 15) + 1
    return time_period

df_rentals['time_period'] = df_rentals.apply(calculate_time_period, axis=1)


In [215]:
def calculate_time_period(row, column):
    day = row['day']
    time = int(row[column].split('-')[0])
    time_period = (day * 96) + (time // 15) + 1
    return time_period

df_rentals['time_period'] = df_rentals.apply(lambda x: calculate_time_period(x, 'departure_time'), axis=1)
df_returns['time_period'] = df_returns.apply(lambda x: calculate_time_period(x, 'arrival_time'), axis=1)

In [216]:
# df_returns.value_counts

In [217]:
df_rentals[df_rentals.day == 0].sort_values(by='time_period')

,station_id,departure_time,day,rentals,time_period
14163,1,0-15,0,1,1
190698,12,15-30,0,1,2
21937,1,15-30,0,1,2
208278,13,45-60,0,1,4
177763,11,45-60,0,1,4
...,...,...,...,...,...
270068,17,1410-1425,0,1,95
437485,27,1410-1425,0,1,95
7482,0,1425-1440,0,1,96
254637,16,1425-1440,0,1,96


In [218]:
df_returns[df_returns.day == 0].sort_values(by='time_period')

,station_id,arrival_time,day,returns,time_period
182286,11,15-30,0,1,2
291611,18,30-45,0,1,3
135812,8,45-60,0,1,4
90601,5,60-75,0,1,5
355993,22,60-75,0,1,5
...,...,...,...,...,...
275721,17,1425-1440,0,1,96
336902,21,1425-1440,0,1,96
135068,8,1425-1440,0,2,96
52022,3,1425-1440,0,1,96


In [219]:
df_rentals.to_csv("rentals.csv")
df_returns.to_csv("returns.csv")